In [46]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [47]:
# Define dataset path
dataset_dir = "dataset_L_R_F_B_T"  # Replace with your actual dataset path
class_names = sorted([
    d for d in os.listdir(dataset_dir)
    if os.path.isdir(os.path.join(dataset_dir, d))
])

# Get image file paths and labels
file_paths = []
labels = []

for label_index, class_name in enumerate(class_names):
    class_dir = os.path.join(dataset_dir, class_name)
    for fname in os.listdir(class_dir):
        fpath = os.path.join(class_dir, fname)
        if os.path.isfile(fpath) and fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_paths.append(fpath)
            labels.append(label_index)

file_paths = np.array(file_paths)
labels = np.array(labels)


In [48]:
# Stratified train/test split
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, val_idx in splitter.split(file_paths, labels):
    X_train, X_test = file_paths[train_idx], file_paths[val_idx]
    y_train_raw, y_test_raw = labels[train_idx], labels[val_idx]

print("Train class distribution:", np.bincount(y_train_raw))
print("Val class distribution:", np.bincount(y_test_raw))

Train class distribution: [31 20 50 56 59]
Val class distribution: [ 7  5 13 14 15]


In [49]:
# Preprocess image files to array
def preprocess_images(paths, target_size=(600, 600)):
    images = []
    for path in paths:
        img = load_img(path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        images.append(img_array)
    return np.array(images)

In [50]:
X_train_images = preprocess_images(X_train)
X_test_images = preprocess_images(X_test)

In [51]:
# Load MobileNet base model
base_model = EfficientNetB7(input_shape=None,
                       include_top=True,
                       weights='imagenet')
base_model.trainable = True

In [52]:
# Add custom layers
x = base_model.output
x = Dense(128, activation='relu')(x)
predictions = Dense(len(class_names), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [53]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import time
start_time = time.time()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train_images, y_train_raw, epochs=1000, batch_size=4, validation_split=0.2, callbacks=[early_stopping])
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")

Epoch 1/1000
43/43 ━━━━━━━━━━━━━━━━━━━━ 1153s 25s/step - accuracy: 0.2603 - loss: 1.6085 - val_accuracy: 0.4318 - val_loss: 1.6047
Epoch 2/1000
43/43 ━━━━━━━━━━━━━━━━━━━━ 4413s 104s/step - accuracy: 0.4301 - loss: 1.6021 - val_accuracy: 0.3409 - val_loss: 1.6010
Epoch 3/1000
43/43 ━━━━━━━━━━━━━━━━━━━━ 6294s 149s/step - accuracy: 0.5220 - loss: 1.5895 - val_accuracy: 0.3409 - val_loss: 1.5972
Epoch 4/1000
43/43 ━━━━━━━━━━━━━━━━━━━━ 5892s 140s/step - accuracy: 0.4525 - loss: 1.5624 - val_accuracy: 0.3864 - val_loss: 1.5882
Epoch 5/1000
43/43 ━━━━━━━━━━━━━━━━━━━━ 5151s 122s/step - accuracy: 0.4884 - loss: 1.5266 - val_accuracy: 0.5909 - val_loss: 1.5471
Epoch 6/1000
43/43 ━━━━━━━━━━━━━━━━━━━━ 5889s 140s/step - accuracy: 0.4705 - loss: 1.5188 - val_accuracy: 0.6136 - val_loss: 1.5225
Epoch 7/1000
36/43 ━━━━━━━━━━━━━━━━━━━━ 7:17 63s/step - accuracy: 0.4942 - loss: 1.4989

In [ ]:
# Evaluate model
import time
start_time = time.time()
loss, accuracy = model.evaluate(X_test_images, y_test_raw)
end_time = time.time()

total_time = end_time - start_time
# Per image inference time
num_images = len(X_test_images)
time_per_image = total_time / num_images

print(f"Inference completed in {total_time:.2f} seconds for {num_images} images.")
print(f"Average inference time per image: {time_per_image * 1000:.2f} ms")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 945s 7s/step - accuracy: 0.2894 - loss: 1.5368  
Inference completed in 945.40 seconds for 54 images.
Average inference time per image: 17507.41 ms
Test Accuracy: 27.78%


In [ ]:
# Save model
model.save("mobilenet_L_R.h5")
# Load model (optional step, useful for confirmation)
model = tf.keras.models.load_model("mobilenet_L_R.h5")


ValueError: Unknown layer: 'LayerScale'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
# Predict on test set
y_pred_probs = model.predict(X_test_images)
y_pred = np.argmax(y_pred_probs, axis=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 22s 8s/step


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test_raw, y_pred, labels=range(len(class_names)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
disp.plot(cmap=plt.cm.Blues, values_format="d")
plt.title("Confusion Matrix")
plt.xlabel('Predicted Value')
plt.ylabel('True Value')
plt.show()


KeyboardInterrupt: 

<Figure size 600x600 with 0 Axes>

<Figure size 640x480 with 0 Axes>